In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.tasak_reader import TasakReader

# functions for creating database and collecting collocations
from collect_functions_2_ma import *

# Tabel 2 andmete kogumine



**Ülesande püstitus**

Kõik laused, mis sisaldavad mõnda sama tabeli kolmandal lehel (MA) olevat lemmat, mille otseste alluvate hulgas on ma-infinitiiv (ainult ma-vorm, mitte mas, mast, mata, maks). Samamoodi esimeses veerus lause, teises veerus otsitava verbi lemma ja kolmandas veerus ma-infinitiivis verbi lemma.



**Kirjeldus**

Tasakaalus korpusest kogutakse kokku järgmisele mallile vastavad esinemised:

* peaverbi alluvate seas ma-infinitiiv: alluv on verb ja morf kategooriate seal on <code>sup</code>;
  


**Tulemus**

Tulmuseks on tabel sqlite formaadis.

Tabeli veerud
||||
|---|---|---|
|**verb**| verbi lemma |---|
|**verb_compound**| afiksaaladverbid |---|
|**verb_is_aux**| verbil või mõnel verbi alluval on deprel == 'aux'|---|
|**verb_feats**| verbi morf kategooriad |---|
|**verb_deprel**| verbi deprel |---|
|**sup_verb**| verbile alluva ma-infinitiivis verbi lemma |---|
|**sup_verb_feats**| morf kategooriad|---|
|**sup_verb_deprel**| morf kategooriad|---|
|**sentence**| lause tekst |---|
|**sentence_id**| lause id andmebaasis|---|


Lisaks kogutakse ka verbide statistika.

Kogutakse eraldi verbi esinemissagedust ja "sobiva" verbi sagedust.

**Filtreeritud verbidega tabel**

Filtreeritud verbidega tabel savelstatakse tsv-formaadis.
Tabelisse salvestatakse [verbs_2_ma.tsv](verbs_2_ma.tsv) nimekirja alusel.




In [ ]:
%%time

file_name = 'data/tasak.vert'

my_reader = TasakReader(
   file_name = file_name
)


!mkdir img_ma/
!rm img_ma/*


In [ ]:
%%time

TYPE = 'ma_laused'
TABLENAME = f'{TYPE}'
BATCH_SIZE = 100000

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"tasak_{TYPE}_{date_time}.sqlite"

my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


# hoitakse algusest lõpuni mälus
verb_global_stat = {}

# kollokatsioonid, tühjendatakse peale igat salvestamist
collocations = []
count = 0
for collection_id, graph in my_reader.get_sentences():
    
    count += 1
    if not collection_id:
        collection_id = count

    #print(f'-------------{collection_id}-----------')
    #print(str(graph))

    
    collocations, verb_global_stat, = extract_something(graph, collection_id, collocations, verb_global_stat )
    



    if not collection_id == 0 and not count % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(collocations, collection_id)
        collocations = []
        
   
    
    
# saving last batch
my_sqlite_db.save_coll_to_db(collocations, collection_id)

In [ ]:

# Salvestatud andmete filtreerimine ja tsv faili salvestamine
import pandas as pd
import sqlite3 as sqlite
df_verbs = pd.read_csv("verbs_2_ma.tsv", sep="\t")
df_verbs.fillna('', inplace=True)
display(df_verbs.head())
verbs = df_verbs['verb'].to_list()


db_conn = sqlite.connect(db_file_name)

df = pd.read_sql('SELECT * FROM da_laused', db_conn)
df['verb_compound_list'] = df.verb_compound.apply(lambda x: x.split(','))

display(df.head())
display(df.shape)

def do_delete(row):
    global verbs
    for cmp in row['verb_compound_list']:
        key = (cmp + ' ' +  row['verb']).strip()
        #if row['verb'] == 'leppima':
        #    print(key)
        if key in verbs:
            #print(key)
            return False
    return True

df['delete'] = df.apply(do_delete, axis=1)

df = df[df['delete'] == False]
del df['verb_compound_list']
del df['delete']
display(df.shape)
df.to_csv(db_file_name+'_final.tsv', index=None) 

In [ ]:
verbs